In [1]:
#dependencies and set up
import pandas as pd 
from bs4 import BeautifulSoup as bs
import requests 
from splinter import Browser
import time
from urllib.parse import urljoin
#set up Chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [2]:
#visit the nasa website with chromedriver
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

#create the soup
html = browser.html
nasa_soup = bs(html, 'html.parser')

In [3]:
time.sleep(1)

In [4]:
latest_article = nasa_soup.find('li', class_='slide')
latest_title = latest_article.find('div', class_='content_title').text
news_p = latest_article.a.text
print(latest_title)
print(news_p)

Tricky Terrain: Helping to Assure a Safe Rover Landing
How two new technologies will help Perseverance, NASA’s most sophisticated rover yet, touch down onto the surface of Mars this month.Tricky Terrain: Helping to Assure a Safe Rover Landing


In [5]:
# articles = nasa_soup.find('li', class_='slide')
# for article in articles: 
#     title = article.find('div', class_='content_title').text
#     p = article.a.text
# #     latest_paragraph = article.
#     print(f'Title: {title} \n')
#     print(f'Paragraph: {p}')
#     print('_______________')

In [6]:
browser.quit()

In [7]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [8]:
#Visit JPL website
# jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
# NOTE: JPL's website redesigned where images are located... the url below is the new location to find latest mars featured images
jpl_url = 'https://mars.nasa.gov/multimedia/images/?page=0&per_page=25&order=pub_date+desc&search=&condition_1=1%3Ais_in_resource_list&category=51'
browser.visit(jpl_url)
#create soup for jpl site 
time.sleep(2)
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

#navigate to the featured image's page 
item = jpl_soup.find('ul', class_='item_list')
latest_item = item.li
pic_href = latest_item.a['href']
browser.click_link_by_href(pic_href)
#create soup for the new page and store image url in new variable
jpl_image_page = browser.html
jpl_image_page_soup = bs(jpl_image_page, 'html.parser') 
image_anchor = jpl_image_page_soup.find('a', class_='fancybox')
image_link = image_anchor.find('img')['src']

#use urljoin to get full image url
base = 'https://mars.nasa.gov'
featured_image_url = urljoin(base, image_link)
featured_image_url

C:\Users\im022\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:474: FutureWarning: browser.find_link_by_href is deprecated. Use browser.links.find_by_href instead.
  warnings.warn(


'https://mars.nasa.gov/system/resources/detail_files/25564_spinoff-1200.jpg'

In [9]:
#load page for mars facts 
mars_facts = 'https://space-facts.com/mars/'
browser.visit(mars_facts)
facts_html = browser.html

In [10]:
# scrape mars facts using pandas
facts = pd.read_html(facts_html)
table = facts[0]
#write table to html 
table_html = table.to_html(index=False, index_names=False)


In [11]:
table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [2]:
#visit usgs site 
usgs = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs)
usgs_html = browser.html 
usgs_soup = bs(usgs_html, 'html.parser')
    


In [3]:
#make list of links to visit 
links = usgs_soup.find_all('div', class_='item')
links_list = []
for link in links:
    links_list.append(link.h3.text)
links_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [4]:
hemisphere_urls = []
for link in links_list:
    browser.click_link_by_partial_text(link)
    new_url= browser.html
    new_soup = bs(new_url, 'html.parser')
    container = new_soup.find('div', class_='downloads')
    sample = container.find('li')
    href = sample.find('a')['href']
    info_dict = dict({"title":link, "img_url":href})
#     print(info_dict)
    hemisphere_urls.append(info_dict)
    browser.visit(usgs)

C:\Users\im022\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [5]:
#check to see if all info was appended to hemisphere_urls 
hemisphere_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [16]:
# Mongo

In [29]:
for i in range(len(hemisphere_urls)): 
    print(str(hemisphere_urls[i]['img_url']))

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [28]:
hemisphere_urls[0]['img_url']

'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'

In [34]:
image_link = browser.html
new_soup = bs(image_link, 'html.parser')

In [38]:
new_soup.find_all('img')

[<img alt="USGS: Science for a Changing World" class="logo" height="60" src="/images/usgs_logo_main_2x.png"/>,
 <img alt="NASA" class="logo" height="65" src="/images/logos/nasa-logo-web-med.png"/>,
 <img alt="PDS Cartography and Imaging Science Node" class="logo" height="65" src="/images/pds_logo-invisible-web.png"/>,
 <img alt="Astropedia" src="/images/astropedia/astropedia-logo-main.png" style="width:200px;border:none;float:right;"/>,
 <img class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/>,
 <img class="wide-image" src="/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"/>,
 <img alt="Valles Marineris Hemisphere Unenhanced thumbnail" class="thumb description-thumb" src="/cache/images/f789a481f60deaf01996343999491211_valles_marineris_unenhanced.tif_thumb.png"/>,
 <img alt="Valles Marineris Hemisphere Enhanced thumbnail" class="thumb description-thumb" src="/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valle

In [66]:
browser.click_link_by_partial_text('Cerberus Hemisphere Enhanced')
new_url= browser.html
new_soup = bs(new_url, 'html.parser')
# link_stuff = browser.find_link_by_partial_text('Sample').first.click()
# browser.url


In [67]:
container = new_soup.find('div', class_='downloads')
sample = container.find('li')

In [79]:
anchor = sample.find('a')['href']

In [80]:
anchor

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'